In [5]:
import numpy as np
import pandas as pd
import re
import sys
sys.path.insert(1, '../')
from notebooks.preprocess import pfeature_process

#from models.network import create_model

In [2]:
def combine(inputs, labels, new_inputs, new_labels):
    new_inputs = np.vstack(new_inputs)
    new_labels = np.hstack(new_labels)
    min_size = min(inputs.shape[0], new_inputs.shape[0])
    new_inputs = new_inputs[:, :min_size]

    
    inputs = np.vstack((inputs, new_inputs))
    labels = np.hstack((labels, new_labels))

    return inputs, labels


def random_replace(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        num_to_replace = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_replace, replace=False)
        ip[indices] = np.random.choice(np.arange(5, 25), num_to_replace, replace=True)

        new_inputs.append(ip)
        new_labels.append(label)

    return new_inputs, new_labels


def random_delete(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        ip = list(ip[:unpadded_len])
        num_to_delete = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_delete, replace=False)
        for i in reversed(sorted(indices)):
            ip.pop(i)
        ip.extend([0] * (200 - len(ip)))

        new_inputs.append(np.asarray(ip))
        new_labels.append(label)

    return new_inputs, new_labels


def random_replace_with_A(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        num_to_replace = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_replace, replace=False)
        ip[indices] = m2['A']

        new_inputs.append(ip)
        new_labels.append(label)

    return new_inputs, new_labels


def random_swap(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        ip = list(ip[:unpadded_len])
        num_to_swap = round(unpadded_len * factor)
        indices = np.random.choice(range(1, unpadded_len, 2), num_to_swap, replace=False)
        for i in indices:
            ip[i-1], ip[i] = ip[i], ip[i-1]
        ip.extend([0] * (200 - len(ip)))

        new_inputs.append(np.asarray(ip))
        new_labels.append(label)

    return new_inputs, new_labels


def random_insertion_with_A(inputs, labels, factor):
    new_inputs = []
    new_labels = []
    for idx in range(inputs.shape[0]):
        ip = inputs[idx]
        label = labels[idx]

        try:
            unpadded_len = np.where(ip == 0)[0][0]
        except IndexError:
            unpadded_len = len(ip)
        ip = list(ip[:unpadded_len])
        num_to_insert = round(unpadded_len * factor)
        indices = np.random.choice(unpadded_len, num_to_insert, replace=False)
        for i in indices:
            ip.insert(i, m2['A'])
        if len(ip) < 200:
            ip.extend([0] * (200 - len(ip)))
        elif len(ip) > 200:
            ip = ip[:200]

        new_inputs.append(np.asarray(ip))
        new_labels.append(label)

    return new_inputs, new_labels


def random_masking(sequences, mask_prob=0.15, mask_token_id=0):
    masked_sequences = np.copy(sequences)
    mask = np.random.rand(*sequences.shape) < mask_prob
    masked_sequences[mask] = mask_token_id
    return masked_sequences

In [6]:
seqs = pd.read_csv('../data/raw/vegf_augmented.csv')
seqs

,Sequence,Label
0,RRKPGRGKRRREQKRP,1
1,RRPKRGGKRRERKQRPDCKPRR,1
2,RRKPGRKGRRREQKPRDSPKRR,1
3,RRPKRGGKRRERKQRPDAVPRR,1
4,KCGRKGRCREKQRPDSKPRR,1
...,...,...
2949,ACSHSGWGCGHGSWSCGRRRRRRRR,0
2950,ACHGRRWGCGRHRGRCG,0
2951,GLFGCGRKKRRQRRRPPQ,0
2952,KRRRRRRRRRE,0


In [4]:
inputs = seqs['Sequence'].to_numpy()
labels = seqs['Label'].to_numpy()

In [109]:
inputs.shape

(240,)

In [122]:
 # new_inputs1, new_labels1 = random_replace(inputs, labels, 0.02)
# new_inputs2, new_labels2 = random_delete(inputs, labels, 0.02)
# new_inputs3, new_labels3 = random_replace_with_A(inputs, labels, 0.02)
new_inputs4, new_labels4 = random_swap(inputs, labels, 0.04)
# new_inputs5, new_labels5 = random_insertion_with_A(inputs, labels, 0.02)
#new_inputs6, new_labels6 = random_masking(inputs, mask_prob=0.15, mask_token_id=0)


C:\Users\silvalej\AppData\Local\Temp\ipykernel_10236\936724860.py:88: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  unpadded_len = np.where(ip == 0)[0][0]


In [123]:
new_inputs4 = np.vstack(new_inputs4)
new_inputs = pd.DataFrame(new_inputs4 )
augmented = new_inputs.apply(lambda row: re.sub(r'[0]','',''.join(row.values.astype(str))), axis=1)




In [124]:
augmented.shape

(264,)

In [125]:
augmented = pd.DataFrame(augmented, columns = ['Sequence'])

In [126]:
augmented = pd.concat([augmented, pd.DataFrame(new_labels4, columns=['Label'])], axis = 1)
augmented

,Sequence,Label
0,RRKPGRGKRRREQKRP,1
1,RRPKRGGKRRERKQRPDCKPRR,1
2,RRKPGRKGRRREQKPRDSPKRR,1
3,RRPKRGGKRRERKQRPDAVPRR,1
4,KCGRKGRCREKQRPDSKPRR,1
...,...,...
259,KGRGRKAAEKQAPSDKPRR,1
260,AGRGARAAEAQRSPDKPRR,1
261,RRPKRGGKRRREKQRPSDAAAR,1
262,KSVRGKGKGQKRKRKKRSYK,1


In [127]:
augmented.rename(columns={0:'Sequence', 0:'Label'}, inplace = True)
augmented

,Sequence,Label
0,RRKPGRGKRRREQKRP,1
1,RRPKRGGKRRERKQRPDCKPRR,1
2,RRKPGRKGRRREQKPRDSPKRR,1
3,RRPKRGGKRRERKQRPDAVPRR,1
4,KCGRKGRCREKQRPDSKPRR,1
...,...,...
259,KGRGRKAAEKQAPSDKPRR,1
260,AGRGARAAEAQRSPDKPRR,1
261,RRPKRGGKRRREKQRPSDAAAR,1
262,KSVRGKGKGQKRKRKKRSYK,1


In [128]:
seqs.shape

(24, 2)

In [129]:
augmented = pd.concat([augmented, seqs], axis = 0, ignore_index=True)
augmented

,Sequence,Label
0,RRKPGRGKRRREQKRP,1
1,RRPKRGGKRRERKQRPDCKPRR,1
2,RRKPGRKGRRREQKPRDSPKRR,1
3,RRPKRGGKRRERKQRPDAVPRR,1
4,KCGRKGRCREKQRPDSKPRR,1
...,...,...
283,KGRGKRAAEKQAPSDKPRR,1
284,AGRGARAAEAQRPSDKPRR,1
285,RRPKGRGKRRREKQRPSDAAAR,1
286,KSVRGKGKGQKRKRKKSRYK,1


In [134]:
augmented.to_csv('./data/vegf_augmented.csv')

In [130]:
inputs = augmented['Sequence'].to_numpy()
labels = augmented['Label'].to_numpy()

In [8]:
with open('../data/processed/VEGF_augmented.fasta','w', encoding='UTF8') as f:
    for _, row in seqs.iterrows():
        f.write(f">{row['Label']}\n{row['Sequence']}\n")

In [9]:
#Ejecutar cd-hit en colab, para windows usar el comando con wsl
#! cd-hit -i data/processed/VEGF.fasta -o data/processed/VEGF.txt -c 0.99

import subprocess

# Define the WSL command buscar 
command = ["wsl", "cd-hit", "-i", "../data/processed/VEGF_augmented.fasta", "-o", "../data/processed/VEGF_augmented.txt", "-c", "0.99"]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and errors
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)


STDOUT: ================================================================
Program: CD-HIT, V4.8.1 (+OpenMP), Aug 20 2021, 08:39:56
Command: cd-hit -i ../data/processed/VEGF_augmented.fasta -o
         ../data/processed/VEGF_augmented.txt -c 0.99

Started: Thu Feb 20 13:32:26 2025
                            Output                              
----------------------------------------------------------------
total seq: 2954
longest and shortest : 97 and 11
Total letters: 62742
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90452245


comparing sequences from          0  to       2954
..
     2954  finished       2938  clusters

Approximated maximum memory consumption: 76M
writing new database
writing clustering information
p

In [10]:
vegf_data = pfeature_process('../data/processed/VEGF_augmented.txt', '../data/processed/vegf_augmented_processed.csv')